# AFIB Detection Using 1D CNN on Cleaned ECG Dataset

This notebook trains a binary AFIB detector using the cleaned and balanced dataset generated by data_handling.ipynb.

Workflow:
1. Setup paths and load cleaned mapping
2. Verify data and prepare labels
3. Split dataset (train/val/test)
4. Compute global channel statistics
5. Define ECG dataset class
6. Create data loaders
7. Define and train 1D CNN model
8. Evaluate on test set


## 1. Imports


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

print("✓ All imports successful")


## 2. Configuration & Path Setup


In [ ]:
# Hyperparameters
DOWNSAMPLE = 1  # Downsample factor (e.g., 2 = keep every 2nd sample)
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths (all relative to project root)
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
CLEANED_ROOT = DATA_DIR / "cleaned_balanced_AFIB_SR"
CLEANED_WFDB_DIR = CLEANED_ROOT / "WFDBRecords"
MAPPING_CSV = CLEANED_ROOT / "file_mapping_cleaned_with_folds.csv"

# Model save path (main_2)
MODEL_DIR = PROJECT_ROOT / "cnn_models"/"500Hz"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
MODEL_PATH = MODEL_DIR / "ecg_1dcnn_afib_balanced_500Hz.pth"

print(f"Device: {DEVICE}")
print(f"MAPPING_CSV: {MAPPING_CSV}")
print(f"MODEL_PATH: {MODEL_PATH}")
print("\n✓ Configuration complete")


## 3. Load Cleaned Mapping & Verify Data


In [ ]:
# Load the cleaned mapping
cleaned_mapping = pd.read_csv(MAPPING_CSV)

print(f"Loaded {len(cleaned_mapping):,} records from {MAPPING_CSV.name}")
print(f"\nColumns: {list(cleaned_mapping.columns)}")

# Verify required columns exist
required_cols = ['record_id', 'ecg_path', '_AFIB', '_SR', 'fold']
missing_cols = [col for col in required_cols if col not in cleaned_mapping.columns]
assert not missing_cols, f"Missing required columns: {missing_cols}"
print(f"✓ All required columns present: {required_cols}")

# Basic statistics
print(f"\n--- Dataset Statistics ---")
print(f"Total records: {len(cleaned_mapping):,}")
print(f"AFIB records (_AFIB=1): {cleaned_mapping['_AFIB'].sum():,}")
print(f"Non-AFIB records (_AFIB=0): {(cleaned_mapping['_AFIB'] == 0).sum():,}")
print(f"SR records (_SR=1): {cleaned_mapping['_SR'].sum():,}")
print(f"Non-SR records (_SR=0): {(cleaned_mapping['_SR'] == 0).sum():,}")

# Display sample rows
print("\nSample rows:")
cleaned_mapping.head()


## 4. Prepare Labels for Binary AFIB Detection


In [ ]:
# Use _AFIB as the primary target column
target_col = "_AFIB"

# Ensure target is strictly 0/1
cleaned_mapping[target_col] = pd.to_numeric(cleaned_mapping[target_col], errors='coerce').fillna(0)
cleaned_mapping[target_col] = (cleaned_mapping[target_col] > 0).astype(int)

print(f"Target column: {target_col}")
print("Target distribution:")
print(cleaned_mapping[target_col].value_counts().sort_index())
print("\n✓ Labels prepared for binary AFIB detection")


## 5. Split Dataset (Train/Val/Test)


In [ ]:
# Split dataset using precomputed patient-level folds (no leakage)
all_folds = sorted(cleaned_mapping['fold'].unique())
test_folds = [0, 1]  # outer test folds (~40% here because 2/5); adjust as needed
train_folds = [f for f in all_folds if f not in test_folds]
val_fold = train_folds[-1]  # use the last training fold for validation
train_only_folds = [f for f in train_folds if f != val_fold]

train_mask = cleaned_mapping['fold'].isin(train_only_folds)
val_mask = cleaned_mapping['fold'] == val_fold
test_mask = cleaned_mapping['fold'].isin(test_folds)

idx_train = cleaned_mapping.index[train_mask].to_numpy()
idx_val = cleaned_mapping.index[val_mask].to_numpy()
idx_test = cleaned_mapping.index[test_mask].to_numpy()

y_train = cleaned_mapping.loc[idx_train, target_col].to_numpy()
y_val = cleaned_mapping.loc[idx_val, target_col].to_numpy()
y_test = cleaned_mapping.loc[idx_test, target_col].to_numpy()

print("--- Dataset Splits (by fold) ---")
print(f"All folds: {all_folds}")
print(f"Train folds: {train_only_folds}")
print(f"Val fold: {val_fold}")
print(f"Test folds: {test_folds}")
print(f"Train: {len(idx_train):,} samples (AFIB: {y_train.sum():,}, Non-AFIB: {(y_train == 0).sum():,})")
print(f"Val:   {len(idx_val):,} samples (AFIB: {y_val.sum():,}, Non-AFIB: {(y_val == 0).sum():,})")
print(f"Test:  {len(idx_test):,} samples (AFIB: {y_test.sum():,}, Non-AFIB: {(y_test == 0).sum():,})")
print("\n✓ Dataset split complete (fold-based)")


## 6. Compute Global Channel Statistics


In [ ]:
def compute_channel_stats(mapping, indices, downsample=None):
    """
    Compute global per-channel mean and std from a subset of training data.
    Stacks up to 512 random samples to estimate statistics.
    """
    if len(indices) > 512:
        sample_indices = np.random.choice(indices, size=512, replace=False)
    else:
        sample_indices = indices

    data_list = []

    for idx in sample_indices:
        row = mapping.iloc[int(idx)]
        npy_file = Path(row["ecg_path"])
        if not npy_file.exists():
            continue
        try:
            data = np.load(npy_file)
            # Ensure 2D array and shape (12, L)
            if data.ndim != 2:
                continue
            if data.shape[0] > data.shape[1]:
                data = data.T
            if data.shape[0] != 12:
                continue
            if downsample is not None and downsample > 1:
                data = data[:, ::downsample]
            data_list.append(data)
        except Exception:
            continue

    if not data_list:
        raise ValueError("No valid data loaded for computing statistics")

    # Align lengths to stack
    min_len = min(arr.shape[1] for arr in data_list)
    data_list = [arr[:, :min_len] for arr in data_list]
    stack = np.stack(data_list, axis=0)  # (N, 12, T)

    mean = stack.mean(axis=(0, 2)).astype(np.float32)
    std = (stack.std(axis=(0, 2)) + 1e-6).astype(np.float32)
    return mean, std

print("✓ Function defined: compute_channel_stats")


In [ ]:
# Compute global statistics from training set
print("Computing global channel statistics from training data...")
global_mean, global_std = compute_channel_stats(
    cleaned_mapping,
    idx_train,
    downsample=DOWNSAMPLE,
)

print(f"\nGlobal mean (per channel): {global_mean}")
print(f"Global std (per channel):  {global_std}")
print("\n✓ Channel statistics computed")


## 7. Define ECG Dataset Class


In [ ]:
class ECGDataset(Dataset):
    """PyTorch Dataset for ECG data that loads from .npy files."""
    def __init__(self, mapping, indices, downsample=None, target_col="_AFIB",
                 channel_mean=None, channel_std=None):
        self.mapping = mapping.iloc[indices].reset_index(drop=True)
        self.downsample = downsample
        self.target_col = target_col
        self.channel_mean = channel_mean if channel_mean is not None else np.zeros(12, dtype=np.float32)
        self.channel_std = channel_std if channel_std is not None else np.ones(12, dtype=np.float32)

    def __len__(self):
        return len(self.mapping)

    def __getitem__(self, idx):
        row = self.mapping.iloc[idx]
        record_id = str(row["record_id"])
        # Resolve path robustly: support relative paths stored in CSV
        npy_file = Path(row["ecg_path"])
        if not npy_file.is_absolute():
            npy_file = (PROJECT_ROOT / npy_file).resolve()
        try:
            data = np.load(npy_file)
            # Ensure 2D array and shape (12, L)
            if data.ndim != 2:
                raise ValueError(f"Expected 2D array, got shape {data.shape}")
            if data.shape[0] > data.shape[1]:
                data = data.T
            if data.shape[0] != 12:
                raise ValueError(f"Expected 12 channels, got {data.shape[0]}")
            if self.downsample is not None and self.downsample > 1:
                data = data[:, ::self.downsample]
            # Normalize
            data = (data - self.channel_mean[:, None]) / self.channel_std[:, None]
        except Exception as e:
            print(f"Error loading {record_id}: {e}")
            # Fallback dummy tensor (kept simple)
            data = np.zeros((12, 2500), dtype=np.float32)
        try:
            tval = float(row[self.target_col])
            label = 1.0 if tval > 0 else 0.0
        except Exception:
            label = 0.0
        data_tensor = torch.from_numpy(data.astype(np.float32))
        label_tensor = torch.tensor(label, dtype=torch.float32)
        return data_tensor, label_tensor

print("✓ ECGDataset class defined")


## 8. Create DataLoaders


In [ ]:
# Create datasets
train_ds = ECGDataset(cleaned_mapping, idx_train, downsample=DOWNSAMPLE, target_col="_AFIB",
                       channel_mean=global_mean, channel_std=global_std)
val_ds = ECGDataset(cleaned_mapping, idx_val, downsample=DOWNSAMPLE, target_col="_AFIB",
                     channel_mean=global_mean, channel_std=global_std)
test_ds = ECGDataset(cleaned_mapping, idx_test, downsample=DOWNSAMPLE, target_col="_AFIB",
                      channel_mean=global_mean, channel_std=global_std)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"Train loader: {len(train_loader)} batches")
print(f"Val loader:   {len(val_loader)} batches")
print(f"Test loader:  {len(test_loader)} batches")
print("\n✓ DataLoaders created")


## 9. Define 1D CNN Model


In [ ]:
class ECG1DCNN(nn.Module):
    def __init__(self, in_channels=12, num_classes=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.conv4 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

model = ECG1DCNN(in_channels=12, num_classes=1).to(DEVICE)
print(f"Model architecture:\n{model}")
print(f"\n✓ Model initialized on {DEVICE}")


## 10. Training Setup


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("Loss function: BCEWithLogitsLoss")
print(f"Optimizer: Adam (lr={LEARNING_RATE})")
print("\n✓ Training setup complete")


## 11. Training Loop


In [ ]:
history = {'train_loss': [], 'val_loss': [], 'val_auc': [], 'val_acc': []}
best_val_auc = 0.0

print(f"Starting training for {NUM_EPOCHS} epochs...\n")
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data).squeeze()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / max(1, len(train_loader))

    model.eval()
    val_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data).squeeze()
            loss = criterion(output, target)
            val_loss += loss.item()
            probs = torch.sigmoid(output).cpu().numpy()
            all_preds.extend(probs)
            all_targets.extend(target.cpu().numpy())
    avg_val_loss = val_loss / max(1, len(val_loader))
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    val_auc = roc_auc_score(all_targets, all_preds) if len(np.unique(all_targets)) > 1 else 0.5
    val_acc = accuracy_score(all_targets, (all_preds > 0.5).astype(int))
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['val_auc'].append(val_auc)
    history['val_acc'].append(val_acc)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val AUC: {val_auc:.4f} | Val Acc: {val_acc:.4f}")
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"  → Best model saved (AUC: {val_auc:.4f})")

print(f"\n✓ Training complete!")
print(f"Best validation AUC: {best_val_auc:.4f}")


## 12. Plot Training History


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend(); axes[0].grid(True)
axes[1].plot(history['val_auc'], label='Val AUC', color='green')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('AUC'); axes[1].set_title('Validation AUC')
axes[1].legend(); axes[1].grid(True)
axes[2].plot(history['val_acc'], label='Val Accuracy', color='orange')
axes[2].set_xlabel('Epoch'); axes[2].set_ylabel('Accuracy'); axes[2].set_title('Validation Accuracy')
axes[2].legend(); axes[2].grid(True)
plt.tight_layout(); plt.show()


## 13. Load Best Model & Evaluate on Test Set


In [ ]:
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

print(f"Loaded best model from {MODEL_PATH}")
print("\nEvaluating on test set...\n")

test_loss = 0.0
all_test_preds, all_test_targets = [], []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(DEVICE), target.to(DEVICE)
        output = model(data).squeeze()
        loss = criterion(output, target)
        test_loss += loss.item()
        probs = torch.sigmoid(output).cpu().numpy()
        all_test_preds.extend(probs)
        all_test_targets.extend(target.cpu().numpy())
avg_test_loss = test_loss / max(1, len(test_loader))
all_test_preds = np.array(all_test_preds)
all_test_targets = np.array(all_test_targets)

test_auc = roc_auc_score(all_test_targets, all_test_preds) if len(np.unique(all_test_targets)) > 1 else 0.5
test_pred_labels = (all_test_preds > 0.5).astype(int)
test_acc = accuracy_score(all_test_targets, test_pred_labels)
conf_matrix = confusion_matrix(all_test_targets, test_pred_labels)

print("--- Test Set Results ---")
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test AUC:  {test_auc:.4f}")
print(f"Test Acc:  {test_acc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(all_test_targets, test_pred_labels, target_names=['Non-AFIB', 'AFIB']))


## 14. Visualize Confusion Matrix


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-AFIB', 'AFIB'],
            yticklabels=['Non-AFIB', 'AFIB'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix (Test Set)\nAUC: {test_auc:.4f}, Acc: {test_acc:.4f}')
plt.tight_layout()
plt.show()

print("\n============================================================")
print("✓ AFIB detection pipeline complete!")
print("============================================================")


## 15. ROC Curve (Receiver Operating Characteristic)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(all_test_targets, all_test_preds)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"ROC-AUC Score: {roc_auc:.4f}")

## 16. DET Curve (Detection Error Tradeoff)

In [ ]:
from sklearn.metrics import DetCurveDisplay

# Create and plot DET curve
fig, ax = plt.subplots(figsize=(10, 8))
DetCurveDisplay.from_predictions(all_test_targets, all_test_preds, ax=ax, name='AFIB Detection')

ax.set_title('Detection Error Tradeoff (DET) Curve', fontsize=14, fontweight='bold')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('False Negative Rate', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()

print("DET Curve: Lower values indicate better performance")
print(f"The curve shows the tradeoff between false positives and false negatives")

# 17. Metrics

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, confusion_matrix, roc_auc_score
)

# Choose source: prefer test-set outputs, fallback to validation
if 'all_test_targets' in globals() and 'all_test_preds' in globals():
    y_true = np.array(all_test_targets)
    y_prob = np.array(all_test_preds)
elif 'all_targets' in globals() and 'all_preds' in globals():
    y_true = np.array(all_targets)
    y_prob = np.array(all_preds)
else:
    print("No predictions found. Run the evaluation cell (Section 13) or the training loop first.")
    y_true = np.array([])
    y_prob = np.array([])

if y_true.size > 0 and y_prob.size > 0:
    # Binary predictions at 0.5 threshold
    y_pred = (y_prob > 0.5).astype(int)

    # Core metrics
    acc = accuracy_score(y_true, y_pred)
    pre = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1  = f1_score(y_true, y_pred, zero_division=0)
    mcc = matthews_corrcoef(y_true, y_pred)

    # Specificity (True Negative Rate) with stable 2x2 matrix
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    TN, FP, FN, TP = cm.ravel()
    spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan

    # ROC-AUC using probabilities
    roc_auc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else 0.5

    print(f"Acc:  {acc:.3f}")
    print(f"F1:   {f1:.3f}")
    print(f"Pre:  {pre:.3f}")
    print(f"Rec:  {rec:.3f}")
    print(f"Spec: {spec:.3f}")
    print(f"MCC:  {mcc:.3f}")
    print(f"AUC:  {roc_auc:.3f}")
else:
    print("Metrics not computed because prediction arrays are empty.")

In [ ]:

# Save results to results folder
from datetime import datetime

# Create results directory if it doesn't exist
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Calculate sample rate (original 500 Hz downsampled by factor DOWNSAMPLE)
original_sample_rate = 500  # Hz
sample_rate = original_sample_rate / DOWNSAMPLE  # Hz

# Generate results file
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_name = "1D-CNN"
results_filename = RESULTS_DIR / f"results_{model_name}_{timestamp}.txt"

# Format results text
results_text = f"""
================================================================================
AFIB Detection Model Results
================================================================================
Model Name:        {model_name}
Sample Rate:       {sample_rate:.1f} Hz (Original: {original_sample_rate} Hz, Downsample: {DOWNSAMPLE}x)
Batch Size:        {BATCH_SIZE}
Learning Rate:     {LEARNING_RATE}
Num Epochs:        {NUM_EPOCHS}
Device:            {DEVICE}
Timestamp:         {timestamp}

================================================================================
Test Set Metrics
================================================================================
Accuracy (Acc):    {acc:.4f}
F1-Score (F1):     {f1:.4f}
Precision (Pre):   {pre:.4f}
Recall (Rec):      {rec:.4f}
Specificity (Spec):{spec:.4f}
Matthews Corr Coef:{mcc:.4f}
ROC-AUC (AUC):     {roc_auc:.4f}

Test Loss:         {avg_test_loss:.4f}

================================================================================
Confusion Matrix
================================================================================
                  Predicted
               Non-AFIB  AFIB
Actual Non-AFIB    {TN:>4d}    {FP:>4d}
       AFIB        {FN:>4d}    {TP:>4d}

TP (True Positive):   {TP}
TN (True Negative):   {TN}
FP (False Positive):  {FP}
FN (False Negative):  {FN}

================================================================================
"""

# Write to file
with open(results_filename, 'w') as f:
    f.write(results_text)

print(f"✓ Results saved to: {results_filename}")
print(results_text)


## 18. Per-Fold Performance Analysis

Evaluate model performance on each individual fold to ensure no patient leakage and assess generalization across different patient groups.

In [ ]:
# Verify no patient leakage between folds
print("=== Verifying Patient-Level Fold Assignment ===\n")

patient_fold_counts = cleaned_mapping.groupby('patient_id')['fold'].nunique()
patients_in_multiple_folds = patient_fold_counts[patient_fold_counts > 1]

print(f"Total unique patients: {cleaned_mapping['patient_id'].nunique():,}")
print(f"Total records: {len(cleaned_mapping):,}")
print(f"Patients appearing in multiple folds: {len(patients_in_multiple_folds)}")

if len(patients_in_multiple_folds) > 0:
    print("\n⚠️ WARNING: Patient leakage detected!")
    print("\nPatients in multiple folds:")
    for patient, count in patients_in_multiple_folds.items():
        folds = cleaned_mapping[cleaned_mapping['patient_id'] == patient]['fold'].unique()
        print(f"  {patient}: appears in {count} folds -> {sorted(folds)}")
else:
    print("\n✅ NO PATIENT LEAKAGE DETECTED")
    print("   Each patient appears in exactly one fold - proper stratification confirmed!")

# Distribution across folds
print("\n=== Fold Distribution Summary ===")
for fold in sorted(cleaned_mapping['fold'].unique()):
    fold_data = cleaned_mapping[cleaned_mapping['fold'] == fold]
    n_records = len(fold_data)
    n_patients = fold_data['patient_id'].nunique()
    n_afib = fold_data['_AFIB'].sum()
    n_sr = (fold_data['_AFIB'] == 0).sum()
    print(f"Fold {fold}: {n_records:4d} records ({n_patients:4d} patients) | AFIB: {n_afib:3d} ({n_afib/n_records*100:.1f}%) | SR: {n_sr:3d} ({n_sr/n_records*100:.1f}%)")

In [ ]:
# Evaluate performance on each fold separately
print("\n=== Per-Fold Performance Evaluation ===\n")

fold_results = []

for fold in sorted(cleaned_mapping['fold'].unique()):
    # Get indices for this fold
    fold_mask = cleaned_mapping['fold'] == fold
    idx_fold = cleaned_mapping.index[fold_mask].to_numpy()
    
    # Create dataset and loader for this fold
    fold_ds = ECGDataset(cleaned_mapping, idx_fold, downsample=DOWNSAMPLE, target_col="_AFIB",
                         channel_mean=global_mean, channel_std=global_std)
    fold_loader = DataLoader(fold_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Evaluate on this fold
    model.eval()
    fold_preds, fold_targets = [], []
    with torch.no_grad():
        for data, target in fold_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data).squeeze()
            probs = torch.sigmoid(output).cpu().numpy()
            fold_preds.extend(probs)
            fold_targets.extend(target.cpu().numpy())
    
    fold_preds = np.array(fold_preds)
    fold_targets = np.array(fold_targets)
    
    # Calculate metrics
    fold_auc = roc_auc_score(fold_targets, fold_preds) if len(np.unique(fold_targets)) > 1 else 0.5
    fold_pred_labels = (fold_preds > 0.5).astype(int)
    fold_acc = accuracy_score(fold_targets, fold_pred_labels)
    fold_cm = confusion_matrix(fold_targets, fold_pred_labels)
    
    # Store results
    fold_results.append({
        'fold': fold,
        'n_samples': len(fold_targets),
        'auc': fold_auc,
        'accuracy': fold_acc,
        'confusion_matrix': fold_cm
    })
    
    # Print results
    print(f"Fold {fold}:")
    print(f"  Samples: {len(fold_targets):4d}")
    print(f"  AUC:     {fold_auc:.4f}")
    print(f"  Accuracy: {fold_acc:.4f}")
    print(f"  Confusion Matrix:")
    print(f"    TN={fold_cm[0,0]:3d}  FP={fold_cm[0,1]:3d}")
    print(f"    FN={fold_cm[1,0]:3d}  TP={fold_cm[1,1]:3d}")
    
    # Calculate additional metrics
    if fold_cm[1,1] + fold_cm[0,1] > 0:
        precision = fold_cm[1,1] / (fold_cm[1,1] + fold_cm[0,1])
    else:
        precision = 0.0
    if fold_cm[1,1] + fold_cm[1,0] > 0:
        recall = fold_cm[1,1] / (fold_cm[1,1] + fold_cm[1,0])
    else:
        recall = 0.0
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print()

# Summary statistics
aucs = [r['auc'] for r in fold_results]
accs = [r['accuracy'] for r in fold_results]

print("=" * 60)
print("SUMMARY STATISTICS ACROSS FOLDS")
print("=" * 60)
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
print(f"Min AUC:       {np.min(aucs):.4f} (Fold {aucs.index(min(aucs))})")
print(f"Max AUC:       {np.max(aucs):.4f} (Fold {aucs.index(max(aucs))})")
print("=" * 60)

In [ ]:
# Visualize per-fold performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot AUC per fold
folds = [r['fold'] for r in fold_results]
aucs = [r['auc'] for r in fold_results]
accs = [r['accuracy'] for r in fold_results]

axes[0].bar(folds, aucs, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axhline(y=np.mean(aucs), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(aucs):.4f}')
axes[0].set_xlabel('Fold', fontsize=12)
axes[0].set_ylabel('AUC', fontsize=12)
axes[0].set_title('AUC per Fold', fontsize=14, fontweight='bold')
axes[0].set_ylim([0.8, 1.0])
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Accuracy per fold
axes[1].bar(folds, accs, color='coral', alpha=0.7, edgecolor='black')
axes[1].axhline(y=np.mean(accs), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(accs):.4f}')
axes[1].set_xlabel('Fold', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Accuracy per Fold', fontsize=14, fontweight='bold')
axes[1].set_ylim([0.8, 1.0])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Per-fold performance analysis complete!")

In [ ]:
# Visualize per-fold performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot AUC per fold
folds = [r['fold'] for r in fold_results]
aucs = [r['auc'] for r in fold_results]
accs = [r['accuracy'] for r in fold_results]

axes[0].bar(folds, aucs, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axhline(y=np.mean(aucs), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(aucs):.4f}')
axes[0].set_xlabel('Fold', fontsize=12)
axes[0].set_ylabel('AUC', fontsize=12)
axes[0].set_title('AUC per Fold', fontsize=14, fontweight='bold')
axes[0].set_ylim([0.8, 1.0])
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Accuracy per fold
axes[1].bar(folds, accs, color='coral', alpha=0.7, edgecolor='black')
axes[1].axhline(y=np.mean(accs), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(accs):.4f}')
axes[1].set_xlabel('Fold', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Accuracy per Fold', fontsize=14, fontweight='bold')
axes[1].set_ylim([0.8, 1.0])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Per-fold performance analysis complete!")

In [ ]:
# Evaluate performance on each fold separately
print("\n=== Per-Fold Performance Evaluation ===\n")

fold_results = []

for fold in sorted(cleaned_mapping['fold'].unique()):
    # Get indices for this fold
    fold_mask = cleaned_mapping['fold'] == fold
    idx_fold = cleaned_mapping.index[fold_mask].to_numpy()
    
    # Create dataset and loader for this fold
    fold_ds = ECGDataset(cleaned_mapping, idx_fold, downsample=DOWNSAMPLE, target_col="_AFIB",
                         channel_mean=global_mean, channel_std=global_std)
    fold_loader = DataLoader(fold_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Evaluate on this fold
    model.eval()
    fold_preds, fold_targets = [], []
    with torch.no_grad():
        for data, target in fold_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data).squeeze()
            probs = torch.sigmoid(output).cpu().numpy()
            fold_preds.extend(probs)
            fold_targets.extend(target.cpu().numpy())
    
    fold_preds = np.array(fold_preds)
    fold_targets = np.array(fold_targets)
    
    # Calculate metrics
    fold_auc = roc_auc_score(fold_targets, fold_preds) if len(np.unique(fold_targets)) > 1 else 0.5
    fold_pred_labels = (fold_preds > 0.5).astype(int)
    fold_acc = accuracy_score(fold_targets, fold_pred_labels)
    fold_cm = confusion_matrix(fold_targets, fold_pred_labels)
    
    # Store results
    fold_results.append({
        'fold': fold,
        'n_samples': len(fold_targets),
        'auc': fold_auc,
        'accuracy': fold_acc,
        'confusion_matrix': fold_cm
    })
    
    # Print results
    print(f"Fold {fold}:")
    print(f"  Samples: {len(fold_targets):4d}")
    print(f"  AUC:     {fold_auc:.4f}")
    print(f"  Accuracy: {fold_acc:.4f}")
    print(f"  Confusion Matrix:")
    print(f"    TN={fold_cm[0,0]:3d}  FP={fold_cm[0,1]:3d}")
    print(f"    FN={fold_cm[1,0]:3d}  TP={fold_cm[1,1]:3d}")
    
    # Calculate additional metrics
    if fold_cm[1,1] + fold_cm[0,1] > 0:
        precision = fold_cm[1,1] / (fold_cm[1,1] + fold_cm[0,1])
    else:
        precision = 0.0
    if fold_cm[1,1] + fold_cm[1,0] > 0:
        recall = fold_cm[1,1] / (fold_cm[1,1] + fold_cm[1,0])
    else:
        recall = 0.0
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print()

# Summary statistics
aucs = [r['auc'] for r in fold_results]
accs = [r['accuracy'] for r in fold_results]

print("=" * 60)
print("SUMMARY STATISTICS ACROSS FOLDS")
print("=" * 60)
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
print(f"Min AUC:       {np.min(aucs):.4f} (Fold {aucs.index(min(aucs))})")
print(f"Max AUC:       {np.max(aucs):.4f} (Fold {aucs.index(max(aucs))})")
print("=" * 60)

In [ ]:
# Verify no patient leakage between folds
print("=== Verifying Patient-Level Fold Assignment ===\n")

patient_fold_counts = cleaned_mapping.groupby('patient_id')['fold'].nunique()
patients_in_multiple_folds = patient_fold_counts[patient_fold_counts > 1]

print(f"Total unique patients: {cleaned_mapping['patient_id'].nunique():,}")
print(f"Total records: {len(cleaned_mapping):,}")
print(f"Patients appearing in multiple folds: {len(patients_in_multiple_folds)}")

if len(patients_in_multiple_folds) > 0:
    print("\n⚠️ WARNING: Patient leakage detected!")
    print("\nPatients in multiple folds:")
    for patient, count in patients_in_multiple_folds.items():
        folds = cleaned_mapping[cleaned_mapping['patient_id'] == patient]['fold'].unique()
        print(f"  {patient}: appears in {count} folds -> {sorted(folds)}")
else:
    print("\n✅ NO PATIENT LEAKAGE DETECTED")
    print("   Each patient appears in exactly one fold - proper stratification confirmed!")

# Distribution across folds
print("\n=== Fold Distribution Summary ===")
for fold in sorted(cleaned_mapping['fold'].unique()):
    fold_data = cleaned_mapping[cleaned_mapping['fold'] == fold]
    n_records = len(fold_data)
    n_patients = fold_data['patient_id'].nunique()
    n_afib = fold_data['_AFIB'].sum()
    n_sr = (fold_data['_AFIB'] == 0).sum()
    print(f"Fold {fold}: {n_records:4d} records ({n_patients:4d} patients) | AFIB: {n_afib:3d} ({n_afib/n_records*100:.1f}%) | SR: {n_sr:3d} ({n_sr/n_records*100:.1f}%)")

## 18. Per-Fold Performance Analysis

Evaluate model performance on each individual fold to ensure no patient leakage and assess generalization across different patient groups.